In [ ]:
# Import Notebook Packages
import warnings
from urllib import request
from urllib.request import urlopen
from urllib.error import HTTPError

import re
from io import StringIO
import os

os.environ["USE_PYGEOS"] = "0"

import geopandas as gpd
import xarray as xr
import pandas as pd
import pathlib as pl
import numpy as np
import pyogrio

import netCDF4

import ipyleaflet

import branca
import branca.colormap as cm

import folium
from folium import Circle, Marker
from folium import plugins
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from ipywidgets import widgets

from ipyleaflet import Map, GeoJSON

# PyPRMS needs
from pyPRMS import Dimensions
from pyPRMS.metadata.metadata import MetaData
from pyPRMS import ControlFile
from pyPRMS import Parameters
from pyPRMS import ParameterFile
from pyPRMS.prms_helpers import get_file_iter, cond_check
from pyPRMS.constants import (
    DIMENSIONS_HDR,
    PARAMETERS_HDR,
    VAR_DELIM,
    PTYPE_TO_PRMS_TYPE,
    PTYPE_TO_DTYPE,
)
from pyPRMS.Exceptions_custom import ParameterExistsError, ParameterNotValidError
import networkx as nx
from collections.abc import KeysView

import pywatershed as pws

from rich.console import Console
from rich.progress import track
from rich.progress import Progress
from rich import pretty

pretty.install()
con = Console()

warnings.filterwarnings("ignore")

#### Adds:
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime as dt
from datetime import datetime

import ipyleaflet
from ipyleaflet import Map, GeoJSON

from folium import Choropleth
from folium.plugins import BeautifyIcon

import branca
import branca.colormap as cm

import plotly.graph_objects as go
import plotly
import plotly.subplots
import plotly.express as px

import dataretrieval.nwis as nwis

import efc
import NHM_helpers

import jupyter_black

jupyter_black.load()

## Introduction
This notebook is designed for the user to setup NHM-Assist and a provided NHM subbasin model in a project workspace.  If you have not been provided a NHM subbasin model, default paths are set to an example model that can be downloaded from the USGS OSN storage pod following these steps.
1. Open up a miniforge prompt.
2. cd to the **NHM-Assist** folder
3. type `python pull_domain.py --name=willamette_river`

## National Hydrologic Model subabsin models
A sub-basin models for any domain within the NHM are extracted using an automated workflow that generates a complete set of PRMS input files that contain the data and parameters required for a NHM-PRMS model [Regan and others, 2018](https://pubs.usgs.gov/publication/tm6B9). This tool is written in the [Python language](https://www.python.org) and is designed to be run from the command line on [USGS high-performance computing resources](https://www.usgs.gov/advanced-research-computing). At this time, users do not need to download this software and instead can request a model subset following these steps:

1. Go to the web page [https://www.sciencebase.gov/catalog/item/5e29b87fe4b0a79317cf7df5](https://www.sciencebase.gov/catalog/item/5e29b87fe4b0a79317cf7df5)
2. Click the child item titled, [“GIS Features of the Geospatial Fabric for the National Hydrologic Model, version 1.1.”](https://www.sciencebase.gov/catalog/item/5e29d1a0e4b0a79317cf7f63)
3. Download attached files "GFv1.1.gdb.zip" and compare NHM segments to your area-of-interest.
4. Send an emial to pnorton@usgs.gov that includes the following:
   * Name, Email address, Organization, and optionally, Phone;
   * Using GFv1.1.gdb, include one or more national model segments (nhm_seg) associated with watershed outlet(s) points in your area-of-interest.
   * Include a short descriptive summary of your modeling application.

## **User Provided Information**
<font size=4>The user must provide and/or review information where <font color='green'>&#x270D;**Enter Information:**</font> appears in this notebook. 

#### &#x270D;<font color='green'>**Enter Information:**</font> **selected NHM subbasin model folder name**.
> The default is set to the example NHM subbasin model name, "20220819_oregon".

In [ ]:
subbasin = "willamette_river"

#### &#x270D;<font color='green'>**Enter Information:**</font> **GIS file format**. 
> The default format is a geopackage (**.gpkg**) but other formats such as ESRI shape file (**.shp**) may have been provided.

In [ ]:
GIS_format = ".gpkg"

#### &#x270D;<font color='green'>**Enter Information:**</font> **parameter file name**. 
> The default file name is **myparam.param**. Parameter files from other model calibrations, i.e. byHRU, byHW, byHWobs can be selected here.

In [ ]:
param_file = "myparam.param"

#### &#x270D;<font color='green'>**Enter Information:**</font> **control file name**. 
> The default file name is **control.default.bandit**.

In [ ]:
control_file_name = "control.default.bandit"

#### &#x270D;<font color='green'>**Enter Information:**</font> **NHD Hydrologic Region of model domain**. 
> Please list the Region(s) that intersect the NHMx domain, for example ["16", "17", "18"] or ["12"]. The default is set to ["17"] for the example NHM subbasin model.

In [ ]:
model_domain_regions = ["17"]  # you can list one, two, or many here

##### ![image.png](https://www.epa.gov/system/files/styles/large/private/images/2021-08/v2webmap-pi.png?itok=yflJml-t)
##### https://www.epa.gov/waterdata/get-nhdplus-national-hydrography-dataset-plus-data

#### All needed information has been provided above. Now the rest of the notebook will create the needed objects, paths and directories for NHM-Assist notebooks.
> You're <font size=5 color="red">**NOT FINISHED**</font> yet!</font> <font size=5 color="red">**SAVE YOUR NOTEBOOK**</font> to retain entered information!

## Workspace Setup
The NHM-Assist repository is designed to be placed in a project folder. Two subfolders in the the NHM-Assist repository have critical supporting documents.

1. The **data_dependencies** folder with needed supporting files not included in the NHM v1.1 data release [Markstrom and others, 2024](https://www.sciencebase.gov/catalog/item/626c0d67d34e76103cd2ce4a)

2. The **data_domain** folder contains the provided NHM subbasin model.
    The provided NHM subbasin model folder should contain:
    - **control.default.bandit**
    - **myparam.param**
    - **sf_data.nc**
    - **cbh.nc**
    - **GIS** folder containing
        - **model_nhru.shp**
        - **model_nsegment.shp**
        - **model_npoigages.shp**
        - and/or **model_layers.gpkg**
          
    NHM-Assist will create additional files and folders in NHM subbasin model folder. These include
    - **default_gages.csv**
    - **NWISgages.csv**
    - **tmin.nc**
    - **tmax.nc**
    - **prcp.nc**
    - **model_output** folder
    - **notebook_output_files** folder containing:
        - **Folium_maps** folder
        - **GeoDataFrames** folder
        - **nc_files** folder
        - **shapefiles** folder
    **Note:** If subfolders do no exist, they will be created when needed.

   



In [ ]:
# Setup Workspace
model_dir = pl.Path(f"domain_data/{subbasin}")
model_dir_txt = f"The selected [bold black]model[/bold black] is [bold black]{model_dir}[/bold black]"

# Establish paths
out_dir = model_dir / "output"
notebook_dir = pl.Path("./").resolve()
NHM_dir = f"{notebook_dir}/data_dependencies/NHM_v1_1"

# This is part of a custom setup for OWRD projects
# Set to pull download streamflow data from Oregon and Washing State databases in Notebook 0c Create Streamflow files.
if (
    ("16" in model_domain_regions)
    or ("17" in model_domain_regions)
    or ("18" in model_domain_regions)
):
    owrd_domain = "yes"
    owrd_domain_txt = f"; the model lies within Washington and/or Oregon State boundaries, and streamflow data from those state databases will be retrieved."
else:
    owrd_domain = "no"
    owrd_domain_txt = f"; the model lies outside Washington State and/or Oregon State boundaries, and streamflow data from USGS NWIS only will be retrieved."

if not pl.Path.exists(
    out_dir
):  # Checks to see if the folder already exists and writes folder if absent
    out_dir_txt = f"          NHMx [bold blue]output folder[/bold blue] --created."
    pl.Path(out_dir).mkdir(
        parents=True, exist_ok=True
    )  # makes the new folder if not there already
else:
    out_dir_txt = f"          NHMx [bold blue]output folder[/bold blue] --exists."

param_filename = f"{model_dir}/{param_file}"

prms_meta = MetaData(version=5, verbose=False).metadata
pdb = ParameterFile(param_filename, metadata=prms_meta, verbose=False)


param_file_txt = (
    f"          model parameter file : [bold black]{param_file}[/bold black]"
)

# Other file path names are hardcoded here for NHM-Assist
nwis_gages_file = model_dir / "NWISgages.csv"
gages_file = model_dir / "gages.csv"  # Provided from OWRD
default_gages_file = model_dir / "default_gages.csv"  # Provided from OWRD
output_netcdf_filename = model_dir / "notebook_output_files" / "nc_files" / "sf_efc.nc"

# Create/verify Jupyter notebooks output folder and subfolders in the model directory.
notebook_output_dir = pl.Path(
    model_dir / "notebook_output_files"
).resolve()  # sets a path variable for the new folder
if not pl.Path.exists(
    notebook_output_dir
):  # Checks to see if the folder already exists and writes folder if absent
    nb_out_dir_txt = (
        f"          [bold black]notebook_output_files folder[/bold black] --created."
    )
    pl.Path(notebook_output_dir).mkdir(
        parents=True, exist_ok=True
    )  # makes the new folder if not there already
else:
    nb_out_dir_txt = (
        f"          [bold black]notebook_output_files folder[/bold black] --exists."
    )

shapefile_dir = pl.Path(
    notebook_output_dir / "shapefiles"
).resolve()  # Directory where shapefiles from Book 1 were written
if not pl.Path.exists(
    shapefile_dir
):  # Checks to see if the folder already exists and writes folder if absent
    shapefile_dir_txt = (
        f"               [bold black]shapefiles folder[/bold black] --created."
    )
    pl.Path(shapefile_dir).mkdir(
        parents=True, exist_ok=True
    )  # makes the new folder if not there already
else:
    shapefile_dir_txt = (
        f"               [bold black]shapefiles folder[/bold black] --exists."
    )

GeoDataFrames_dir = pl.Path(notebook_output_dir / "GeoDataFrames").resolve()
if not pl.Path.exists(
    GeoDataFrames_dir
):  # Checks to see if the folder already exists and writes folder if absent
    GeoDataFrames_dir_txt = (
        f"               [bold black]GeoDataFrames folder[/bold black] --created."
    )
    pl.Path(GeoDataFrames_dir).mkdir(
        parents=True, exist_ok=True
    )  # makes the new folder if not there already
else:
    GeoDataFrames_dir_txt = (
        f"               [bold black]GeoDataFrames folder[/bold black] --exists."
    )

Folium_maps_dir = pl.Path(notebook_output_dir / "Folium_maps").resolve()
if not pl.Path.exists(
    Folium_maps_dir
):  # Checks to see if the folder already exists and writes folder if absent
    Folium_maps_dir_txt = (
        f"               [bold black]Folium_maps folder[/bold black] --created."
    )
    pl.Path(Folium_maps_dir).mkdir(
        parents=True, exist_ok=True
    )  # makes the new folder if not there already
else:
    Folium_maps_dir_txt = (
        f"               [bold black]Folium_maps folder[/bold black] --exists."
    )

##Check to see if the output subdirectory was already made, and if not will make it.
nc_files_dir = pl.Path(notebook_output_dir / "nc_files").resolve()
if not (notebook_output_dir / "nc_files").exists():
    nc_files_dir_txt = (
        f"               [bold black]nc_files[/bold black] folder --created."
    )
    (notebook_output_dir / "nc_files").mkdir(parents=True)
else:
    nc_files_dir_txt = (
        f"               [bold black]nc_files[/bold black] folder --exists."
    )

# Printing the start and End dats for model run
control = pws.Control.load_prms(
    pl.Path(f"domain_data/{subbasin}") / control_file_name, warn_unused_options=False
)
range_txt = f" The model run time and observation retrieval range are {control.start_time} - {control.end_time} and set in the model control file{control_file_name}. If altered, all notebooks must be re-run."

con.print(
    "[bold green]Verify slected model workspace files and settings:",
    "\n",
    model_dir_txt,
    owrd_domain_txt,
    "\n",
    out_dir_txt,
    "\n",
    param_file_txt,
    "\n",
    range_txt,
    "\n",
    # shapefile_dir_txt,
    # "\n",
    # GeoDataFrames_dir_txt,
    # "\n",
    # Folium_maps_dir_txt,
    # "\n",
    # nc_files_dir_txt,
    # "\n",
)